In [2]:
import pandas as pd 
import numpy as np 

In [3]:
from google.colab import files
uploaded = files.upload()

Saving tmdb_5000_credits.csv to tmdb_5000_credits.csv
Saving tmdb_5000_movies.csv to tmdb_5000_movies.csv


In [191]:
credits = pd.read_csv('tmdb_5000_credits.csv')
movies = pd.read_csv('tmdb_5000_movies.csv')
print(credits.shape)
print(movies.shape)

(4803, 4)
(4803, 20)


In [192]:
credits.columns = ['id','tittle','cast','crew']
movies= movies.merge(credits,on='id')
movies.shape

(4803, 23)

In [193]:
C = movies['vote_average'].mean()
C

6.092171559442011

In [194]:
m = movies['vote_count'].quantile(0.85)
m

1300.6999999999998

In [195]:
demograph_movies = movies.copy().loc[movies['vote_count'] >= m]
demograph_movies.shape

(721, 23)

In [196]:
def weighted_rating(a, m=m, C=C):
  v = a['vote_count']
  R = a['vote_average']
  return (v/(v+m) * R) + (m/(m+v) * C)

In [197]:
demograph_movies['score'] = demograph_movies.apply(weighted_rating, axis=1)

In [198]:
demograph_movies = demograph_movies.sort_values('score', ascending=False)

In [199]:
demograph_movies[['title', 'vote_count', 'vote_average', 'score']].head(10)

,title,vote_count,vote_average,score
1881,The Shawshank Redemption,8205,8.5,8.170528
662,Fight Club,9413,8.3,8.031958
3232,Pulp Fiction,8428,8.3,8.004820
65,The Dark Knight,12002,8.2,7.993903
3337,The Godfather,5893,8.4,7.982719
96,Inception,13752,8.1,7.926504
809,Forrest Gump,7927,8.2,7.902889
95,Interstellar,10867,8.1,7.885368
329,The Lord of the Rings: The Return of the King,8064,8.1,7.821125
1990,The Empire Strikes Back,5879,8.2,7.818138


In [200]:
pop= demograph_movies.sort_values('popularity', ascending=False)
pop[['title', 'vote_count', 'vote_average', 'score']].head(10)

,title,vote_count,vote_average,score
546,Minions,4571,6.4,6.331810
95,Interstellar,10867,8.1,7.885368
788,Deadpool,10995,7.4,7.261651
94,Guardians of the Galaxy,9742,7.9,7.687059
127,Mad Max: Fury Road,9427,7.2,7.065679
28,Jurassic World,8662,6.5,6.446755
199,Pirates of the Caribbean: The Curse of the Bla...,6985,7.5,7.278997
82,Dawn of the Planet of the Apes,4410,7.3,7.024898
200,The Hunger Games: Mockingjay - Part 1,5584,6.6,6.504058
88,Big Hero 6,6135,7.8,7.501256


In [201]:
from ast import literal_eval
demograph_movies['genres'] = demograph_movies['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [202]:
demograph_movies[['title', 'vote_count', 'vote_average', 'genres', 'score']].head(10)

,title,vote_count,vote_average,genres,score
1881,The Shawshank Redemption,8205,8.5,"[Drama, Crime]",8.170528
662,Fight Club,9413,8.3,[Drama],8.031958
3232,Pulp Fiction,8428,8.3,"[Thriller, Crime]",8.004820
65,The Dark Knight,12002,8.2,"[Drama, Action, Crime, Thriller]",7.993903
3337,The Godfather,5893,8.4,"[Drama, Crime]",7.982719
96,Inception,13752,8.1,"[Action, Thriller, Science Fiction, Mystery, A...",7.926504
809,Forrest Gump,7927,8.2,"[Comedy, Drama, Romance]",7.902889
95,Interstellar,10867,8.1,"[Adventure, Drama, Science Fiction]",7.885368
329,The Lord of the Rings: The Return of the King,8064,8.1,"[Adventure, Fantasy, Action]",7.821125
1990,The Empire Strikes Back,5879,8.2,"[Adventure, Action, Science Fiction]",7.818138


In [203]:
movies['overview'].head(10)

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
5    The seemingly invincible Spider-Man goes up ag...
6    When the kingdom's most wanted-and most charmi...
7    When Tony Stark tries to jumpstart a dormant p...
8    As Harry begins his sixth year at Hogwarts, he...
9    Fearing the actions of a god-like Super Hero l...
Name: overview, dtype: object

In [204]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [205]:
tfidf = TfidfVectorizer(stop_words='english')

In [206]:
movies['overview'] = movies['overview'].fillna('')

In [207]:
tfidf_overview = tfidf.fit_transform(movies['overview'])
tfidf_overview.shape

(4803, 20978)

In [208]:
from sklearn.metrics.pairwise import linear_kernel

In [209]:
cos_sim = linear_kernel(tfidf_overview, tfidf_overview)

In [210]:
indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()
indices

title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64

In [211]:
def des_recommendations(title, cos_sim=cos_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cos_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:16]
    movie_indices = [i[0] for i in sim_scores]
    return movies['title'].iloc[movie_indices]

In [212]:
des_recommendations('Minions')

506                           Despicable Me 2
221                           Stuart Little 2
1218                           The Guilt Trip
2511                               Home Alone
3944                                  Freeway
2892                        Darling Companion
505     The League of Extraordinary Gentlemen
3188                          Velvet Goldmine
1733                        The Spy Next Door
70                             Wild Wild West
3042                                 The Gift
4726                               The Mighty
3687                              Without Men
4403                           The Jimmy Show
2086           Home Alone 2: Lost in New York
Name: title, dtype: object

In [213]:
des_recommendations('The Dark Knight')

3                         The Dark Knight Rises
428                              Batman Returns
3854    Batman: The Dark Knight Returns, Part 2
299                              Batman Forever
1359                                     Batman
119                               Batman Begins
1181                                        JFK
9            Batman v Superman: Dawn of Justice
2507                                  Slow Burn
210                              Batman & Robin
879                         Law Abiding Citizen
2020                                 The Rookie
205          Sherlock Holmes: A Game of Shadows
4135            Gangster's Paradise: Jerusalema
506                             Despicable Me 2
Name: title, dtype: object

In [214]:
from google.colab import files
uploaded = files.upload()

Saving ratings_small.csv to ratings_small.csv


In [226]:
ratings = pd.read_csv('ratings_small.csv')

In [220]:
pip install surprise

     |████████████████████████████████| 11.8MB 6.1MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1618307 sha256=c1e710c640b1118df58897d3611c8de2f7587324e566a90f5fc3eca288ae0c7b
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [224]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [242]:
reader = Reader()
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5)

{'fit_time': (4.959280729293823,
  4.908865451812744,
  5.007390737533569,
  4.896027565002441,
  4.978327989578247),
 'test_mae': array([0.70033006, 0.69458275, 0.69107933, 0.68209781, 0.68917756]),
 'test_rmse': array([0.9076891 , 0.90028678, 0.89423015, 0.88827743, 0.89646736]),
 'test_time': (0.15013813972473145,
  0.2676548957824707,
  0.14571905136108398,
  0.2578296661376953,
  0.14689135551452637)}

In [243]:
train = data.build_full_trainset()
svd.fit(train)

In [252]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [265]:
svd.predict(1, 1339, 3.5)

Prediction(uid=1, iid=1339, r_ui=3.5, est=2.5793731670273496, details={'was_impossible': False})